In [ ]:
from biomni.agent import A1

# import os
# os.environ.setdefault("NCBI_EMAIL", "me@example.com")

agent = A1()
agent.add_mcp(config_path="./mcp_config.yaml")
print(
    agent.go(
        "Search PubMed for BRCA1 and list two paper titles with PMIDs. Must use pubmed_mcp, search_pubmed and get_article_abstract"
    )
)

In [ ]:
print(agent.go("I need to find FDA active ingredient information for the drug 'ibuprofen'."))